## Netflix movie Rating predictor

in this notebook the MALIS project by Eirik Morken en Gijs Wissing will be described.
The goal of our project was to predict the personal rating Gijs would give to a unseen movie based on his Netflix Viewing history and other open data from sources like IMDB. The biggest challenge of this project is dealing with unlabeled data, as Gijs did not provide any ratings to the movies he has viewed on Netflix. Having unlabeled data is a very common real world problem for data scientists. The way it was solved in this project was using a unsupervised clustering algorithm as well as a cusotm made voting system.

#### The Data
To start the project all the necessary data had to be put together. This provided an initial challenge as the dataset from IMDB is larger than the RAM in our machines can handle directly. On top of this the labels used between IMDB and Netflix are completely different. For this reason we used some iterative line processors to integrate the IMDB data into our working dataset. Below you will find a cell showing an example of one of these line processors. No further code will used during in this notebook as the importing, filtering and cleaning of the data is quite a time-consuming process. For that reason beyond this point a saved cleaned dataset will be imported.

In [ ]:
def readAndFilterIMDBdata():
    tsvList = []
    movieList = []
    with open("ProjectData/title_basics.tsv", encoding='utf-8') as ratingsFile:
        rd = csv.reader(ratingsFile, delimiter="\t",quotechar='"')
        for row in rd:
            if row[1] == "movie" or row[1] == "tvMovie" or row[1] == 'short':
                movieList.append(row)
    return movieList


def readTsvFile(tf):
    tsvList = []
    with open(tf) as ratingsFile:
        rd = csv.reader(ratingsFile, delimiter=",")
        for row in rd:
            tsvList.append(row)
    return tsvList

##### Next the main part of the code is reviewed. The following steps are taken:
  *  All the categorical features are encoded in a 1 of K set
  *  All the numerical features are encoded to be binary as well
  *  The binary features are used to train a Kmodes clustering algorithm.
  *  The clusters are identified using a voting system based on the watched and unwatched criteria
  
*quick note is that the kmodes library can be installed using "pip install kmodes"*

In [1]:
import pandas as pd
import numpy as np
from kmodes.kmodes import KModes
data = pd.read_csv('ProjectData/cleanedData.csv')
data = data.replace('\\N',0)
data = data.fillna(value=0)
data.startYear = data.startYear.values.astype(int)
data.runtimeMinutes = data.runtimeMinutes.values.astype(int)

ModuleNotFoundError: No module named 'pandas'

#### Encoding categorical features
The first step in preparing the data for the kmods algorithm, is the apply 1-of-K encoding on the catagorical features. The only catagorical feature in the dataset in the is the movie genre. The dataset contains in total 27 different genres, which with 1-of-K encoding translates to 27 new features. Each movie can have multiple genres, which is a small complication, but dealth with.

In [ ]:
#Reading 
def readTsvFile(tf):
    tsvList = []
    with open(tf) as ratingsFile:
        rd = csv.reader(ratingsFile, delimiter=",")
        for row in rd:
            tsvList.append(row)
    return tsvList

imdbTable = readTsvFile("ProjectData/cleanedData.csv")


#Removing all IMDB movies with no votes
poppedMoviesWithNoVotes = []
for n in range(len(imdbTable)):
    if imdbTable[n][-1] != "":
        poppedMoviesWithNoVotes.append(imdbTable[n])
imdbTable = poppedMoviesWithNoVotes


def mergeWatchTime():
    #Adding column for mergin 
    for x in range(len(imdbTable)):
        imdbTable[x].append('\\N')
    mergeList = []
    for mov in imdbTable:
        for nflix in netflixLocalData:    
            if nflix [0] == mov[2] or nflix [0] == mov[3]:
                mov[-1] = nflix[1]
                
        mergeList.append(mov)
    return mergeList

mergedWatchTimeList = mergeWatchTime()
mergedWatchTimeList.pop(0)


df = pd.DataFrame(data=mergedWatchTimeList)
df.columns = ['tconst','titleType','primaryTitle','originalTitle','isAdult','startYear','endYear','runtimeMinutes','genres','averageRating','numVotes','watchDate']
genresDf = df['genres'].str.get_dummies(sep=',')#this is a single line that encodes the genre categories
merged = pd.concat([df,genresDf],axis='columns')
#print(merged)



#### Encoding numerical features
In the next cell the dataset is prepared for the predictor. This is necessary to be able to use both numerical and categorical features in the K-prototypes. 

In [3]:
from kmodes.kprototypes import KPrototypes
data = pd.read_csv('ProjectData/mergedWatchTime&GenresEncoding.csv')
data = data.replace('\\N',0)
data = data.rename(columns={'\\N':'noGenre'})
data = data.drop(labels='Unnamed: 0',axis = 1)
data = data.fillna(value=0)
popped = data.pop('isAdult')
data['isAdult'] = popped
data.startYear = data.startYear.values.astype(int)
data.runtimeMinutes = data.runtimeMinutes.values.astype(int)
popped = data.pop('startYear')
data['startYear'] = popped
popped = data.pop('averageRating')
data['averageRating'] = popped
popped = data.pop('numVotes')
data['numVotes']= popped
popped = data.pop('runtimeMinutes')
data['runtimeMinutes'] = popped

          tconst titleType                               primaryTitle  \
0      tt0111161     movie                   The Shawshank Redemption   
1      tt0468569     movie                            The Dark Knight   
2      tt1375666     movie                                  Inception   
3      tt0110912     movie                               Pulp Fiction   
4      tt0109830     movie                               Forrest Gump   
5      tt0068646     movie                              The Godfather   
6      tt1345836     movie                      The Dark Knight Rises   
7      tt0848228     movie                               The Avengers   
8      tt0120689     movie                             The Green Mile   
9      tt0071562     movie                     The Godfather: Part II   
10     tt0169547     movie                            American Beauty   
11     tt2015381     movie                    Guardians of the Galaxy   
12     tt0434409     movie                         

#### "Training" the k-prototypes model
In the next model the k-prototypes model is defined and executed. The K-prototypes algorithm was chosen over the simpler K-modes since our data has both categorical and numerical data. This is a unsupervised clustering algorithm, thus it shouldn't be considered training. Both the K-modes and K- prototypes, used the Hamming distance, which is defined as the number of features that are not alike between rows. The rest of the model functions the same as the Kmeans where the distance between the cluster centroid and the cluster members is optimized.

In [4]:
input = data.iloc[:,7:]
input.iloc[:,:22] = input.iloc[:,:22].astype(int)
input.iloc[:,-4:] = input.iloc[:,-4:].astype(float)
kp = KPrototypes(n_clusters=10, init='Cao', n_init=3, verbose=2)#option are Huang and Cao
clusters = kp.fit_predict(input, categorical=list(range(23)))
data['cluster'] = clusters

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 381, cost: 2689.0
Run 1, iteration: 2/100, moves: 134, cost: 2689.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 518, cost: 2719.0
Run 2, iteration: 2/100, moves: 122, cost: 2719.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 480, cost: 2958.0
Run 3, iteration: 2/100, moves: 133, cost: 2890.0
Run 3, iteration: 3/100, moves: 46, cost: 2890.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 142, cost: 2599.0
Run 4, iteration: 2/100, moves: 0, cost: 2599.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 485, cost: 2557.0
Run 5, iteration: 2/100, moves: 98, cost: 2557.0
Init: initializing centroids
Init: initializing cluste

#### Voting the cluster rating
Before utilizing the result from the clusters to the multiple regression algorithm, the weighting of the new feature has to be applied. The weighing of each cluster was decided by the amount of movies watched in the cluster divided by the total amount of movies. This results in the percentage of movies watch in the cluster. It was debated to use these percentage score to classify the clusters in descending order from best cluster to worst cluster, however it was opted to go for a regression predictor directly on the percentages.


In [ ]:
data['clusPercent'] = np.zeros(len(data.numVotes.values))
for clusNum in range(10):
    clusData = data.loc[data.cluster==clusNum,:]
    totalLen = len(clusData.startYear.values)
    watchedLen = len(clusData.loc[clusData.watchDate!=0,'originalTitle'])
    data.loc[data.cluster == clusNum,'clusPercent'] = watchedLen/totalLen*100
print(data.clusPercent)

#### importing validation data and training regression algorithm
In the following code block a new dataset is imported that was specifically collected for validation. It consists of a small number of movies, that hadn't been released on Netflix at the time we gathered the initial dataset. 1/3 of this dataset was watched by Gijs in the meantime. The first 5 movies in the dataset were the movies Gijs had watched. The results were not amazing from an objective point of view, however subjectively the movies with a high percentage are in general great movies Gijs enjoyed or would probably enjoy watching. To conclude, to properly test the performance of the first algorithm that clusters the movies, there would need to be a larger and better labelled dataset. To test the subjective performance of the algorithm Gijs will need to watch some movies that are suggested to him by the algorithm and provide feedback. He will most definitly do this.


In [ ]:
from sklearn.ensemble import RandomForestRegressor
dataTest = pd.read_csv('ProjectData/testSet.csv')
ranForReg = RandomForestRegressor(random_state=1, n_estimators=10)
ranForReg.fit(input,data.clusPercent)
output=ranForReg.predict(dataTest.iloc[:,8:])
plt.scatter(output,dataTest.index)
for index in range(len(dataTest.index)):
    print(str(dataTest.loc[index,'primaryTitle'])+' has cluster percentage= '+ str(output[index]))
plt.show()

